In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
file_path = '../bar_movement/data/'
currency_pair = 'Eur_Usd'

In [3]:
# df = pd.read_csv(file_path + f'Oanda_{currency_pair}_H4_2022-2023.csv')
df = pd.read_csv(file_path + 'test.csv')
df.Date = pd.to_datetime(df.Date, utc=True)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [27]:
def squeeze(barsdata, length=20, length_kc=20, bb_mult=2.0, kc_mult_high=1.0, kc_mult_mid=1.5, kc_mult_low=2.0):
    # Bollinger bands
    m_avg = barsdata['Mid_Close'].rolling(window=length).mean()
    m_std = barsdata['Mid_Close'].rolling(window=length).std(ddof=0)
    bb_upper = m_avg + bb_mult * m_std
    bb_lower = m_avg - bb_mult * m_std

    # Keltner channel
    tr0 = abs(barsdata['Mid_High'] - barsdata['Mid_Low'])
    tr1 = abs(barsdata['Mid_High'] - barsdata['Mid_Close'].shift())
    tr2 = abs(barsdata['Mid_Low'] - barsdata['Mid_Close'].shift())
    tr = pd.concat([tr0, tr1, tr2], axis=1).max(axis=1)
    range_ma = tr.rolling(window=length_kc).mean()
    kc_upper_high = m_avg + range_ma * kc_mult_high
    kc_lower_high = m_avg - range_ma * kc_mult_high
    kc_upper_mid = m_avg + range_ma * kc_mult_mid
    kc_lower_mid = m_avg - range_ma * kc_mult_mid
    kc_upper_low = m_avg + range_ma * kc_mult_low
    kc_lower_low = m_avg - range_ma * kc_mult_low

    # Squeeze
    low_squeeze = (bb_lower >= kc_lower_low) | (bb_upper <= kc_upper_low) # Black
    mid_squeeze = (bb_lower >= kc_lower_mid) | (bb_upper <= kc_upper_mid) # Yellow
    high_squeeze = (bb_lower >= kc_lower_high) | (bb_upper <= kc_upper_high) # Red

    squeeze_values = np.where(high_squeeze, 'red', np.where(mid_squeeze, 'yellow', np.where(low_squeeze, 'black', 'green')))

    # Momentum
    highest_high = barsdata['Mid_High'].rolling(window=length).max()
    lowest_low = barsdata['Mid_Low'].rolling(window=length).min()
    avg_high_low = (highest_high + lowest_low) / 2
    avg_avg_high_low_sma = (avg_high_low + m_avg) / 2
    diff = barsdata['Mid_Close'] - avg_avg_high_low_sma
    squeeze_momentum = diff.rolling(window=length).apply(lambda x: np.polyfit(np.arange(length), x, 1)[0], raw=True)
    iff_1 = np.where(squeeze_momentum > squeeze_momentum.shift(), 'aqua', 'blue')
    iff_2 = np.where(squeeze_momentum < squeeze_momentum.shift(), 'red', 'yellow')
    squeeze_momentum_color = np.where(squeeze_momentum > 0, iff_1, iff_2)

    return squeeze_values, squeeze_momentum_color

def atr(high, low, close, lookback=14):
    high_low = high - low
    high_close = np.abs(high - close.shift())
    low_close = np.abs(low - close.shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)

    return true_range.rolling(lookback).mean()

def atr_bands(high, low, close, lookback=14, atr_multiplier=3):
    scaled_atr_vals = atr(high, low, close, lookback) * atr_multiplier
    lower_band = close - scaled_atr_vals
    upper_band = close + scaled_atr_vals

    return lower_band, upper_band

In [28]:
df['squeeze_value'], df['squeeze_momentum_color'] = squeeze(df)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [29]:
df.iloc[-20:, [0, -2, -1]]

,Date,squeeze_momentum,squeeze_momentum_color
74767,2023-08-31 04:20:00+00:00,4.767669e-05,blue
74768,2023-08-31 04:25:00+00:00,4.765038e-05,blue
74769,2023-08-31 04:30:00+00:00,4.703383e-05,blue
74770,2023-08-31 04:35:00+00:00,4.424436e-05,blue
74771,2023-08-31 04:40:00+00:00,4.069173e-05,blue
74772,2023-08-31 04:45:00+00:00,3.691353e-05,blue
74773,2023-08-31 04:50:00+00:00,3.129323e-05,blue
74774,2023-08-31 04:55:00+00:00,2.577820e-05,blue
74775,2023-08-31 05:00:00+00:00,1.749624e-05,blue
74776,2023-08-31 05:05:00+00:00,1.143985e-05,blue
